In [ ]:
# Mechanism to mock test submits in MongoDB with random dates.
from bson import ObjectId
from config.database import db_UJ        # UJ mongoDB
from datetime import datetime, timedelta
from pymongo import UpdateOne
import random

class MockTestSubmits:
    def __init__(self):
        self.cts = db_UJ['test_st']

    def random_date_gen(self, simplify=False, total_dates=5):
        def df(dt):
            return dt.strftime("%m-%d")

        now = datetime.now()  # Get current date and time
        dates = []

        # Generate dates for the last 3 days (today and the two previous days)
        for i in range(3):
            date = now - timedelta(days=i)
            dates.append(date)

        # Randomly select 5 items from these dates (with replacement)
        random_items = random.choices(dates, k=random.randint(1, total_dates))
        # If simplify is True, format the dates
        if simplify:
            # Format the dates using your df function
            formatted_items = [df(item) for item in random_items]
            return formatted_items    
        else:
            return random_items
        # Format the dates using your df function
        # formatted_items = [df(item) for item in random_items]
        # return formatted_items

    def update_operations(self,mdb_set):
        cts = db_UJ['test_st']   # create/get a collection
        doc_ids = list(cts.find({},{"_id":1}))       # Get limited test document IDs
        update_operation = []
        for i in doc_ids:
            juice = {'submits':self.random_date_gen(simplify=False, total_dates=15)}
            # Use this to remove all the submits fields
            # Use this to simulate setting the new submits field with mock timestamps
            set_ops = {'$set': juice} if mdb_set else {'$unset': {'submits': ''}}
            update_operation.append(
                UpdateOne(
                {"_id": ObjectId(i['_id'])},
                set_ops
            ))
        return update_operation

    def update_submits(self, mdb_set):
        self.cts.bulk_write(self.update_operations(mdb_set=mdb_set))

    def check_transformed_submits(self):
        original = list(self.cts.find({'submits': {'$exists': True}},{"_id":0,"ph":0,"r_id":0}))
        transformed = [{v1: v2} for d in original for v1, v2 in zip(d.values(), list(d.values())[1:])]
        return transformed

In [12]:
import re

from routes.route import flight_aware_w_auth
n_pattern = re.compile("^N[a-zA-Z0-9]{5}$")
n_pattern.match('N12377')

# flight_aware_w_auth()

c:\Users\ujasv\OneDrive\Desktop\base\cirrostrats-backend\venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


<re.Match object; span=(0, 6), match='N12377'>

In [20]:
# from routes.route import raw_search_handler
# x = await raw_search_handler('N63323')

from routes.root.dep_des import Pull_flight_info
from routes.root.flight_deets_pre_processor import response_filter
from routes.root.root_class import Fetching_Mechanism, Source_links_and_api


async def flight_aware_w_auth(flight_number):
    fm = Fetching_Mechanism(flt_num=flight_number)
    sl = Source_links_and_api()
    flt_info = Pull_flight_info()

    # link = sl.aviation_stack(flight_number)
    link = sl.flight_aware_w_auth(flight_number)
    resp_dict: dict = await fm.async_pull([link])
    resp = response_filter(resp_dict, "json",)
    fa_return = resp['flights']
    flight_aware_data = flt_info.fa_data_pull(pre_process=fa_return)

    # Accounted for gate through flight aware. gives terminal and gate as separate key value pairs.
    return flight_aware_data


In [22]:
x = await flight_aware_w_auth('UAL4558')
# x = await flight_aware_w_auth('N787MG')
x

null_flightaware_attrs
Received flight aware data as pre_process
null_flightaware_attrs


{'fa_ident_icao': 'GJS4558',
 'fa_origin': 'KGSO',
 'fa_destination': 'KEWR',
 'fa_registration': 'N551GJ',
 'fa_date_out': '20250606',
 'fa_scheduled_out': '1530Z',
 'fa_estimated_out': '1530Z',
 'fa_scheduled_in': '1721Z',
 'fa_estimated_in': '1655Z',
 'fa_terminal_origin': None,
 'fa_terminal_destination': 'C',
 'fa_gate_origin': None,
 'fa_gate_destination': None,
 'fa_filed_altitude': 'FL350',
 'fa_filed_ete': 4200,
 'fa_route': 'QUAK8 SBV CREWE QUART PHLBO4',
 'fa_sv': 'https://skyvector.com/?fpl=%20KGSO%20QUAK8%20SBV%20CREWE%20QUART%20PHLBO4%20KEWR'}

In [41]:
from pathlib import Path
import pickle


def getting_the_path_right(f_name):
    
    currentWorking = Path.cwd()
    file_path = currentWorking / f_name
    
    return file_path

bulk_flight_deets_path = getting_the_path_right(r"mockTestDataFull.pkl")
# bulk_flight_deets_path = getting_the_path_right(r"example_flight_deet_full_packet.pkl")       # Old legacy Django

bulk_flight_deets = pickle.load(open(bulk_flight_deets_path, 'rb'))
flightStatsMock = {}
newbfd = {}
flightViewMock = {}
jms_STDDS_clearance = {
    "towerAircraftID": "UAL2480",
    "timestamp": "2025-05-10T15:10:03.243000",
    "destinationAirport": "KSFO",
    "clearance": "003 UAL2480\t7737\tKLAX A320/L\tP1540 239\t280 KLAX SUMMR2 STOKD SERFR SERFR4 KSFO             CLEARED SUMMR2 DEPARTURE STOKD TRSN   CLB VIA SID EXC MAINT 5000FT EXP 280 5 MIN AFT DP,DPFRQ 125.2     ",
    "towerBeaconCode": "7737",
    "version_created_at": "2025-05-10T15:11:32.611000"
}
jms_FDPS_base = {
    "flightID": "UAL2075",
    "timestamp": "2025-05-10T15:51:23.088000",
    "organization": "UAL",
    "aircraftType": "A320",
    "registration": "N445UA",
    "departure": "KLAX",
    "arrival": "KSFO",
    "route": "KLAX.SUMMR2.STOKD..SERFR.SERFR4.KSFO/1646",
    "assignedAltitude": "28000.0",
    "currentBeacon": "7737",
    "version_created_at": "2025-05-10T15:52:10.527000"
}

for a,b in bulk_flight_deets.items():
    if 'flightStats' in a:
        flightStatsMock[a]=b
        # bulk_flight_deets.pop(a)
        # print(a,b)
    elif 'flightView' in a:
        flightViewMock[a] =b
    elif a in jms_STDDS_clearance.keys():
        #  print(a,b)
        pass
    elif a in jms_FDPS_base.keys():
        pass
    else:
        newbfd[a]=b
newbfd


{'nas_departure_affected': {'Ground Delay': {'Departure': 'KDEN',
   'Reason': 'thunderstorms',
   'Average Delay': '48 minutes',
   'Maximum Delay': '1 hour and 46 minutes'}},
 'nas_destination_affected': {'Ground Delay': {'Departure': 'KDEN',
   'Reason': 'thunderstorms',
   'Average Delay': '48 minutes',
   'Maximum Delay': '1 hour and 46 minutes'}},
 'dep_weather': {'D-ATIS': 'DEN ARR INFO L 1953Z. 27025G33KT 10SM FEW080 SCT130 SCT200 01/M19 A2955 (TWO NINER FIVE FIVE) RMK AO2 PK WND 29040/1933 SLP040. LLWS ADZYS IN EFCT. HAZUS WX INFO FOR CO, KS, NE, WY AVBL FM FLT SVC. PIREP 30 SW DEN, 2005Z B58T RPRTD MDT-SVR, TB, BTN 14THSD AND 10 THSD DURD. PIREP DEN AREA,, 1929Z PC24 RPRTD MDT, TB, BTN AND FL 190 DURD. EXPC ILS, RNAV, OR VISUAL APCH, SIMUL APCHS IN USE, RWY 25, RWY 26. NOTICE TO AIR MISSION. S C DEICE PAD CLOSED. DEN DME OTS. BIRD ACTIVITY VICINITY ARPT. ...ADVS YOU HAVE INFO L.',
  'METAR': 'KDEN 012054Z 16004KT 1/2SM R05L/P6000FT BR OVC004 08/08 A2978 RMK AO2 SFC VIS 3 SLP0

In [ ]:
from routes.root.tests.mock_test_data import Mock_data


md = Mock_data()
md.flight_data(html_injected_weather=False)

{'towerAircraftID': 'UAL2480',
 'timestamp': '2025-05-10T15:51:23.088000',
 'destinationAirport': 'KSFO',
 'clearance': '003 UAL2480\t7737\tKLAX A320/L\tP1540 239\t280 KLAX SUMMR2 STOKD SERFR SERFR4 KSFO             CLEARED SUMMR2 DEPARTURE STOKD TRSN   CLB VIA SID EXC MAINT 5000FT EXP 280 5 MIN AFT DP,DPFRQ 125.2     ',
 'towerBeaconCode': '7737',
 'version_created_at': '2025-05-10T15:52:10.527000',
 'flightID': 'UAL2075',
 'organization': 'UAL',
 'aircraftType': 'A320',
 'registration': 'N445UA',
 'departure': 'KLAX',
 'arrival': 'KSFO',
 'route': 'KLAX.SUMMR2.STOKD..SERFR.SERFR4.KSFO/1646',
 'assignedAltitude': '28000.0',
 'currentBeacon': '7737',
 'flightStatsDestination': 'KIAH',
 'flightStatsFlightID': 'UA492',
 'flightStatsOrigin': 'KEWR',
 'flightStatsScheduledArrivalTime': '08:31 CDT',
 'flightStatsScheduledDepartureTime': '05:30 EDT',
 'flightViewArrivalGate': '\nD - D1A\n',
 'flightViewDeparture': 'EWR',
 'flightViewDepartureGate': '\nC - C103\n',
 'flightViewDestination': '

In [ ]:
# Changing weather key names from upper case to lower for weathers.
bulk_flight_deets['dep_weather'] = {
"datis": bulk_flight_deets['dep_weather']['D-ATIS'],
"metar": bulk_flight_deets['dep_weather']['METAR'],
"taf": bulk_flight_deets['dep_weather']['TAF'],}

# Introducing/interjecting html code within weather.
weather = Weather_parse()
bulk_flight_deets['dep_weather'] = weather.processed_weather(
    mock_test_data=bulk_flight_deets['dep_weather'])

# Just mocking/copying departure weather into destination as well 
bulk_flight_deets['dest_weather'] = bulk_flight_deets['dep_weather']

return bulk_flight_deets

# IFR and LIFR weather for departure and destination.
ind = path_to_be_used + r"raw_weather_dummy_dataKIND.pkl"
ord = path_to_be_used + r"raw_weather_dummy_dataKORD.pkl"
with open(ind, 'rb') as f:
    dep_weather = pickle.load(f)
with open(ord, 'rb') as f:
    dest_weather = pickle.load(f)

# Injesting the html/css for highlighting here.
weather = Weather_parse()
bulk_flight_deets['dep_weather'] = weather.processed_weather(
    dummy=dep_weather)
weather = Weather_parse()
bulk_flight_deets['dest_weather'] = weather.processed_weather(
    dummy=dest_weather)


data_return = pickle_imports_and_processing()
return data_return



In [1]:
from routes.route import get_search_suggestions


# x = await get_search_suggestions(email='', query='pa', limit=1000)


c:\Users\ujasv\OneDrive\Desktop\base\cirrostrats-backend\venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
import itertools
import string

# Generate all combinations of length 2
alphabets = string.ascii_lowercase[:5]
combinations = [''.join(c) for c in itertools.combinations(alphabets, 2)]
permutations = [''.join(p) for p in itertools.permutations(alphabets, 2)]

print(combinations)
print(f"Total combinations: {len(combinations)}", f"Total permut: {len(permutations)}")

['ab', 'ac', 'ad', 'ae', 'bc', 'bd', 'be', 'cd', 'ce', 'de']
Total combinations: 10 Total permut: 20


In [ ]:
mts = MockTestSubmits()
# mts.update_operations(mdb_set=False)
# mts.update_submits(mdb_set=False)


In [53]:
ctrs = db_UJ['test_rst']   # create/get a collection
submitTerm = "asdf"
timestamp = datetime.now()  # Get current date and time
ctrs.update_one(
    {"rst": submitTerm},
    {"$push": {"submits": timestamp}},
    upsert=True
)

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000001c0'), 'opTime': {'ts': Timestamp(1747987089, 18), 't': 448}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1747987089, 18), 'signature': {'hash': b'\t^\xce\x93pp\x7f\x1b\x90\xd2\x8b\xf9\x9f6\x19O\x11\xec\xb9O', 'keyId': 7460207444094877709}}, 'operationTime': Timestamp(1747987089, 18), 'updatedExisting': True}, acknowledged=True)

In [ ]:
# list(ctrs.find({}).limit(5))
# ctrs.delete_many({})

DeleteResult({'n': 3, 'electionId': ObjectId('7fffffff00000000000001c0'), 'opTime': {'ts': Timestamp(1747992429, 3), 't': 448}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1747992429, 3), 'signature': {'hash': b'w\xf4\x98J\xde\xba\x98Hso\xa8-EN7m\x95.\x12`', 'keyId': 7460207444094877709}}, 'operationTime': Timestamp(1747992429, 3)}, acknowledged=True)

In [ ]:
a = ctrs.aggregate([
    { "$match": { "submits": { "$exists": True} } },
    { "$unwind": "$submits" },
    { "$project": {
        "_id": 0,
        "rst": 1,
        "timestamp": "$submits"
    }}
])
# list(a)
ctrs.find_one({})


In [ ]:
cts_returns =  list(mts.cts.aggregate([
        { "$match": { "submits": { "$exists": True} } },
        { "$unwind": "$submits" },
        { "$addFields": { "timestamp": "$submits" } },
        { "$unset": ["_id", "r_id", "ph", "submits"] }
    ]))

In [ ]:
list(mts.cts.find({"submits": {"$exists": True}}))